In [3]:
%run /Users/cristianswift/Desktop/armbrust-lab/Seaflow-Machine-Learning/python/03_data-QAQC.ipynb

Exception: File `'/Users/cristianswift/Desktop/armbrust-lab/Seaflow-Machine-Learning/Baseline/python/02_data-cleaning.ipynb'` not found.

Exception: File `'/Users/cristianswift/Desktop/armbrust-lab/Seaflow-Machine-Learning/Baseline/python/02_data-cleaning.ipynb'` not found.

# Developing a Random Forest Regressor for Each Population to Predict Biomass (pgC per L)

## Functions:

In [23]:
#function to make numpy array
def to_np(pop_feat):
    pop_feat = np.array(pop_feat)
    return pop_feat



## Seperating dataframes for individual populations

In [24]:
tn413.population.unique()

array(['picoeuk', 'prochloro', 'synecho'], dtype=object)

In [25]:
import numpy as np
#function to make numpy array
def to_np(pop_feat):
    pop_feat = np.array(pop_feat)
    return pop_feat

In [34]:
tn413['PopulationName'] = (
    tn413['population']
    .replace({'picoeuk' : 'non_pro_syn'})
)
tn413.drop('population',axis=1, inplace=True)

In [27]:
pro_tn413 = tn413[tn413.PopulationName == 'prochloro']
syn_tn413 = tn413[tn413.PopulationName == 'synecho']
non_tn413 = tn413[tn413.PopulationName == 'non_pro_syn']


pop_tn413_list = [pro_tn413, syn_tn413, non_tn413]
# creating the labels for all of the populations for biomass (using numpy array so we can predict on them)

label_pro = np.array(pro_tn413.biomass)
label_syn = np.array(syn_tn413.biomass)
label_non = np.array(non_tn413.biomass)

# Creating the features for all of the populations

#these are the dropped features
droped_features = ['time','PopulationName',
                   'biomass', 'lat', 'lon']

features_pro = pro_tn413.drop(droped_features, axis = 1)
features_syn = syn_tn413.drop(droped_features, axis = 1)
features_non = non_tn413.drop(droped_features, axis = 1)


# Saving feature names for later use (doesn't matter which dataframe we use since they all have same column names)
feature_list = list(features_pro.columns)
# Convert to numpy array
pop_features = [features_pro,features_syn,features_non]


features_pro = to_np(features_pro)
features_syn = to_np(features_syn)
features_non = to_np(features_non)

#creating labels to see how the model did
label_pro = np.array(pro_tn413.biomass)
label_syn = np.array(syn_tn413.biomass)
label_non = np.array(non_tn413.biomass)

In [28]:
#loading packages for random forest modeling

from sklearn.model_selection import train_test_split
import joblib